<a href="https://colab.research.google.com/github/johnlukespeight/ML.Practice/blob/main/BassHead.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import jv  # Bessel function of the first kind
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib.colors import LinearSegmentedColormap
import gc
from ipywidgets import IntSlider, interact

# Helper function to get user input with a default value
def get_input(prompt, default):
    user_input = input(f"{prompt} [{default}]: ")
    return float(user_input) if user_input else default

# 1. Change the working directory
new_directory = input("Enter the new working directory path [/Users/cj/Documents/drumcalcs]: ") or "/Users/cj/Documents/drumcalcs"
os.makedirs(new_directory, exist_ok=True)
os.chdir(new_directory)
print("New working directory:", os.getcwd())

# 2. User-defined parameters with suggested default values
B1 = get_input("Enter the surface tension (B1) (N/m)", 100)
B2 = get_input("Enter the areal mass density (B2) (kg/m^2)", 0.2)
B3 = get_input("Enter the radius of the membrane (B3) (m)", 1)
B4 = get_input("Enter the order of the Bessel function (B4)", 0)
radial_step = get_input("Enter the radial step (m)", 0.01)
phi_step = get_input("Enter the phi step (radians)", 0.03)
fps = get_input("Enter the frames per second (FPS) for the animation", 30)

# 3. Calculated parameters
B6 = 1.0  # Amplitude (constant)
bessel_coefficients = [2.4048, 5.5201, 8.6537, 11.7915]

print(f"Calculated parameters based on Bessel coefficients: {bessel_coefficients}")

# 4. Generate phi and radius values based on user inputs
phi_values = np.arange(0, 2 * np.pi + phi_step, phi_step)
radius_values = np.arange(0, B3 + radial_step, radial_step)

print(f"Phi values: {phi_values}")
print(f"Phi step: {phi_step}")
print(f"Radius values: {radius_values}")
print(f"Radial step: {radial_step}")

# 5. Create a custom colormap
colors = [(0, 0.5, 1), (1, 0, 0)]  # Light blue to red
n_bins = 100  # Discretize the colormap
cmap_name = 'custom_cmap'
custom_cmap = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)

# 6. Function to simulate actual displacement data
def actual_displacement_function(B6, B4, bessel_coefficients, B1, B2, B3, radius, phi, t):
    total_displacement = np.zeros_like(radius)
    for B5 in bessel_coefficients:
        C2 = B5 / B2  # Wavenumber
        C3 = (B1 * B5) / (2 * np.pi * B3)  # Modal frequency
        displacement = B6 * jv(B4, C2 * radius) * np.cos(B4 * phi) * np.cos(2 * np.pi * C3 * t)
        total_displacement += displacement
    return total_displacement

# 7. Create meshgrid for phi and radius
Phi, Radius = np.meshgrid(phi_values, radius_values)

# 8. Time steps for the animation
time_steps = np.arange(0, 10, 0.1)  # Extended range for full animation

# 9. Initialize figure and axis for the animation
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')

# Initial plot setup
X = Radius * np.cos(Phi)
Y = Radius * np.sin(Phi)

# Function to update the plot for each frame
def update(t):
    print(f"Generating frame for t={t:.2f} s")  # Debugging statement
    Z = actual_displacement_function(B6, B4, bessel_coefficients, B1, B2, B3, Radius, Phi, t)
    ax.clear()
    surf = ax.plot_surface(X, Y, Z, cmap=custom_cmap)
    ax.set_xlim(-B3, B3)
    ax.set_ylim(-B3, B3)
    ax.set_zlim(-1, 1)
    ax.set_title(f'Drum Head Displacement at t={t:.2f} s')

    # Adding text box for input parameters and fps
    textstr = (f'Surface tension (B1): {B1} N/m\n'
               f'Areal mass density (B2): {B2} kg/m^2\n'
               f'Radius of the membrane (B3): {B3} m\n'
               f'Order of the Bessel function (B4): {B4}\n'
               f'Bessel coefficients: {bessel_coefficients}\n'
               f'Radial step: {radial_step:.4f} m\n'
               f'Phi step: {phi_step:.4f} radians\n'
               f'FPS: {fps}')
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text2D(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=10,
              verticalalignment='top', bbox=props)

    return surf,

# 10. Create the animation
ani = FuncAnimation(fig, update, frames=time_steps, blit=False)
animation_name = "drum_head_animation_with_superimposed_modal_frequencies_memory_management_speedback4.gif"
ani.save(animation_name, writer='pillow', fps=fps)
print(f"Animation generated and saved as {animation_name} at {fps} fps")
plt.close(fig)  # Close the figure to release memory
gc.collect()  # Force garbage collection

# Add interactive control for FPS
def interactive_animation(fps):
    ani = FuncAnimation(fig, update, frames=time_steps, blit=False)
    ani.save(animation_name, writer='pillow', fps=fps)
    print(f"Animation updated and saved as {animation_name} at {fps} fps")

fps_slider = IntSlider(value=fps, min=1, max=60, step=1, description='FPS:')
interact(interactive_animation, fps=fps_slider)




